## Introduction to Bayesian & Frequentist Methodologies w/FAQs

- Bayesian methodologies assume the data is fixed and the parameters are variable - aka we are going to model this data but not be 100% sure about the coefficients.
- Frequentist methodologies assume the data is variable and the parameters are fixed - aka we are going to assume this model is a sample and get as close as we can to the exact mean value of the coefficient.
1. Does Bayesian provide better/different insights for econometric modeling? When is the right scenario to use that versus statsmodels?
    - It provides more depth around your coefficients and their uncertainty/range. Bayesian models shine with smaller data where more complexity is needed.
2. Are there times where you DONT need to bother with Bayesian analysis?
    - It tends to be slower and more resource intensive so when you are running an MVP and want quick results, go with statsmodels
3. Does it work better for a particular target than others? (thinking about revenue vs conversion)
    - It does not work particularly better for a target type, but it might help us better model revenue as we can graph the proper distribution vs the linear assumption that comes with OLS. I also feel like we have a good binary system w/logistic regression in statsmodels (revenue is my answer but it's subjective)
4. Any limitations with the number variables you use in a Bayesian model, or is it {gif}
    - How much time do you have?
5. Are results the same? If not, why? If not, which one is correct or more accurate?
    - The larger the data, the larger the MCMC samples, the closer your mean of your distribution should match the coefficient of the frequentist regression. Neither are more correct or accurate.
6. As we will be wrapping them within a script so we might not get to see the underlying function but which function did you find more intuitive to use?
    - I'm so used to frequentist that it makes more sense, but if you get used to the bayesian way, it can be more elegant
7. What does the PYMC output summary look like? Which of the two summaries did you find to be more useful?
    - Once again, I'm used to the statsmodels output so I find that more intuitive, but I like the visualizations PYMC/Arviz supplies.
8. What are the added advantages of PYMC, if any?
    - Instead of the binary thoughts of stat sig, we can give them a distribution of values where the true value lies. PYMC over other bayesian libraries - PYMC is in python and actively kept up

## PYMC Bayesian Regression - Binary Target

In [ ]:
# ==================================================== #
# Optional to Standardize Variables                    #
# ==================================================== #

# pros: sampling efficiency (NUTS), numerical stability (MCMC), you can transform back to normal at the end (or not - interpret the standard deviation)
# I saw it go from 48 seconds to 16 seconds
# x_norm = (x - x.mean()) / x.std() # x was created above!
x_norm = x.copy()

In [ ]:
%%time
# Naming your pm.Model and opening it up in the context editor to add and change parameters - could code this up differently but not recommended
with pm.Model() as the_model:
    
    # ==================================================== #
    # Prior probabilities for the intercept & coefficients #
    # ==================================================== #
    
    # priors
    # mu - the location or center or mean of your data - if you think your coef is + then make it positive, otherwise you can make it negative
    # sigma - the standard deviation or spread of your data - a large sd would indicate strong confidence in your prior, whereas a small sd indicates uncertainty
    # examples: mu=2, sigma=1, strong priors; mu=1 sigma=10, weak priors; mu=0, sigma=100, uninformative priors
    # Use priors to input domain knowledge, regularlization to avoid overfitting, and make small data more robus
    
    # intercept: represents the predicted value of the dependent variable when all independent variables are equal to zero
    # mu is 0 because we do not have a prior about the intercept
    # sigma is 10 because we have lots of uncertainty about the prior
    # the first variable is the python name which can be used to reference the variable
    # the second variable in quotes is PYMC variable name which is a label you'll see in your plots
    intercept = pm.Normal('intercept',mu=0,sigma=10)
    
    # Betas - even if you have features that are binary, you model them all with normal distributions
    # This is because it's the variable that is binary, not the coefficient
    beta_var1 = pm.Normal('beta_var1',mu=0,sigma=10)
    beta_var2 = pm.Normal('beta_var2',mu=0,sigma=10)
    beta_var3 = pm.Normal('beta_var3',mu=0,sigma=10)
    
    # Another way of doing this if you have a lot of features
    # beta = pm.Normal('beta', mu=0, sigma=10, shape=X.shape[1] - 1)
    # logits = intercept + pm.math.dot(X.iloc[:, 1:], beta)
        # or this...
    # import pytensor.tensor as at
    # july_cpm_cert_drops = pm.Model(coords={"predictors": columns}) # outside of the pm.Model() context
    # beta = pm.Normal("beta", 0, 10, dims="predictors")
    # beta0 = pm.Normal("beta0", 0, 10)
    # mu = beta0 + at.dot(X_train, beta)
    
    
    # ==================================================== #
    # Logistic Likelihood                                  #
    # ==================================================== #
    
    # Regression Formula
    logits = (intercept
              + beta_var1 * x_norm['var1']
              + beta_var2 * x_norm['var2']
              + beta_var3 * x_norm['var3']
             )
    
    # Dependent variable (Bernoulli Distribution)
    y_obs = pm.Bernoulli('y_obs', logit_p=logits, observed=y)
    
    
    # ==================================================== #
    # MCMC NUTS Process                                    #
    # ==================================================== #
    
    # Magic Inference Button
    # 2000 samples from the posterior distribution (2000-5000 is good, closer to 100000 is great)
    # More is better - the samples build up over time as it explores the space - only negative is time and resources
    # Increase when you have a complex model or want precision
    # Decrease when you don't have time or are prototyping
    # trace = pm.sample(4000, return_inferencedata=True)
    
    # When to use multiple chains
    trace = pm.sample(4000, chains=4, return_inferencedata=True)
    # 1. Diagnostics for convergence -> they should overlap in the trace plots
    # 2. Avoiding local minima -> The chains are exploring a posterior space and can get caught in local regions of space
    # 3. Poor mixing happens when MCMC struggles to move across parameter space, multiple chains helps here
    # 4. Improves effectiveness of small sample sizes, improves autocorrelation and improves quality of inferences
    # Use for complex models, diagnositcs, uncertain priors, multi-modal posteriors

In [ ]:
# After sampling, you can convert the coefficients back to the original scale:
# If needed, you can back-transform them for interpretability.
# beta_var1_original = trace['beta_var1'].mean() * x['var1'].std()

In [ ]:
# ==================================================== #
# Visualize Model Results & Diagnostics                #
# ==================================================== #

# Visualize Trace which gives us posterior distributions for the intercept and each beta coefficient

# Left #
#------#
# Posterior distribution plots show the range of values for each coefficient which can help us understand the uncertainty around the causal effect of the variable
# You want to see a normal distribution bell curve - shows the model converged properly; Skewed or multimodal is no good
# Wider distributions shows more uncertainty (narrow=more certainty, but they can be too narrow...)

# Right #
#-------#
# Trace plots: value of sampled parameter at each step of the MCMC process - look for a fuzzy catapillar
# After the inital "burn-in" period, it should look noisy but stable horizontal band - meaning it converged & is exploring the posterior distribution effectively
# It should not show any drift, it should be on the line of the mean
# It should also show good mixing of different values
# Sometimes you disgard the early samples that are part of that burn-in period (person does this, so ask him)

# Signals #
#---------#
# Trace plots that are sticky or deviating off the mean
# Trace plots with multiple chains that don't overlap
# Trade plots that are multi-modal
# Too wide or too narrow posterior distributions

# Diagnostics #
#-------------#
# Gelman-Rubin Stat (R-hat) -> check if chains have converged, < 1.1 means chains have converged
# Effective Sample Size -> How many ind samples the MCMC algorithm has drawn, if ESS is smaller than totla samples, then poor mixing
# Check for autocorrelation of MCMC samples
# Trace plots for multiple chains -> set chains=4 on the pm.sample function & check if the chains overlap for good convergence

az.plot_trace(trace);
# plt.show();
# while this may not look fantastic...google what a bad one looks like and you'll see this is perfectly a-okay

In [ ]:
az.plot_posterior(trace);

In [ ]:
# ==================================================== #
# Summarize Model Results                              #
# ==================================================== #
# Summary of the posterior distributions
# The 95% HDI shows where the bulk of the poterior mass lies
az.summary(trace, hdi_prob=0.95)

In [ ]:
# Graph out model structure
# pm.model_to_graphviz()

In [ ]:
# Posterior Predictive Check
ppc = pm.sample_posterior_predictive(trace, model=)

In [ ]:
# Compare observed data to simulated posterior data
sns.histplot(y, color='blue', label='Observed', kde=True)
sns.histplot(ppc.posterior_predictive.y_obs.mean(axis=0), color='orange', label='Posterior Predictive', kde=True)
plt.legend()
plt.show()

In [ ]:
# Pair Plot (Joint Posterior Distributions)
az.plot_pair(trace, kind="kde", marginals=True);

In [ ]:
# Rank Plot
az.plot_rank(trace)
plt.show()

In [ ]:
# Posterior Interval Plot (HDI Plot)
az.plot_forest(trace);

In [ ]:
# LOO (Leave One Out) Plot
# help(az.loo)
# loo = az.loo(trace,
# az.plot_loo_pit(loo)
# plt.show()

In [ ]:
az.plot_energy(trace)
plt.show()

In [ ]:
# az.plot_cdf(trace)
az.plot_density(trace)
plt.show()

In [ ]:
# az.plot_corr(trace)
az.plot_autocorr(trace)
plt.show()